ETL - Transport Urbain : Chicago & Philadelphie
================================================
Projet : Dashboard Power BI - Analyse Ridership
Auteur : Data Analyst
Date : Février 2026

In [54]:
# Importation des libraries nécessaires
import pandas as pd
import numpy as np
from scipy import stats


### Exploration du fichier Chicago (Excel) : CTA Daily Boarding Totals (multi-mode : Bus, Rail, Total)

In [ ]:
import pandas as pd

df_chicago_mode = pd.read_excel(r"C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\chicago_excel.xlsx")

df_chicago_mode.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,2001-01-01,U,297192,126455,423647
1,2001-01-02,W,780827,501952,1282779
2,2001-01-03,W,824923,536432,1361355
3,2001-01-04,W,870021,550011,1420032
4,2001-01-05,W,890426,557917,1448343


In [56]:
df_chicago_mode['service_date'] = pd.to_datetime(df_chicago_mode['service_date'])

In [57]:
df_chicago_mode = df_chicago_mode.dropna(subset=['service_date', 'bus', 'total_rides'])

In [58]:
df_chicago_mode['day_type'].value_counts()

W    6355
U    1449
A    1296
Name: day_type, dtype: int64

In [59]:
df_chicago_mode.duplicated(subset=['service_date']).sum()

0

In [60]:
df_chicago_mode['year'] = df_chicago_mode['service_date'].dt.year
df_chicago_mode['month'] = df_chicago_mode['service_date'].dt.month

In [61]:
df_chicago_mode = df_chicago_mode[
    (df_chicago_mode['service_date'].dt.year >= 2019) &
    (df_chicago_mode['service_date'].dt.year <= 2025)
]

In [62]:
df_chicago_mode['service_date'].dt.year.value_counts()

2020    366
2024    366
2019    365
2021    365
2022    365
2023    365
2025    334
Name: service_date, dtype: int64

In [ ]:
df_chicago_mode.to_csv(r"C:\Python_challenges\Brief_jury_blanc\output\cta_daily_ridership_clean.csv")

In [64]:
df_chicago_mode.head()

,service_date,day_type,bus,rail_boardings,total_rides,year,month
6574,2019-01-01,U,248879,245852,494731,2019,1
6575,2019-01-02,W,591006,573542,1164548,2019,1
6576,2019-01-03,W,664442,627781,1292223,2019,1
6577,2019-01-04,W,668812,628514,1297326,2019,1
6578,2019-01-05,A,444434,348257,792691,2019,1


In [ ]:
df_chicago_route = pd.read_csv(r"C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\rdf_csv\CTA_Ridership_Total_Fusionne.csv")
df_chicago_route.head()

C:\Users\leila\AppData\Local\Temp\ipykernel_31148\1333771814.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chicago_route = pd.read_csv(r"C:\Python_challenges\Brief_jury_blanc\fusion_finale\CTA_Ridership_Total_Fusionne.csv")


,date,daytype,rides,route
0,2019-01-01,U,3562,47
1,2019-01-02,W,6597,47
2,2019-01-03,W,7768,47
3,2019-01-04,W,8721,47
4,2019-01-05,A,6312,47


In [66]:
df_chicago_route['date'] = pd.to_datetime(df_chicago_route['date'])

In [67]:
df_chicago_route['Calendar_Month'] = df_chicago_route['date'].dt.month

### Philadelphie (CSV) : Import et standardisation.

In [ ]:
import pandas as pd

file_route = r"C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\Philadelphia_csv.zip"
file_mode = r"C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\Philadelphia_csv.zip"

df_phil_route = pd.read_csv(file_route)
df_phil_mode = pd.read_csv(file_mode)

In [69]:
df_phil_route['city'] = 'Philadelphia'
df_phil_mode['city'] = 'Philadelphia'

In [70]:
MODES_AUTORISES = ["Bus", "Heavy Rail", "Regional Rail"]

df_phil_mode = df_phil_mode[df_phil_mode["Mode"].isin(MODES_AUTORISES)]

In [71]:
df_phil_mode['Mode'].value_counts()

Bus              82
Heavy Rail       82
Regional Rail    82
Name: Mode, dtype: int64

In [72]:
colonnes_a_supprimer = ['ObjectId', 'Source']
df_phil_mode = df_phil_mode.drop(columns=[c for c in colonnes_a_supprimer if c in df_phil_mode.columns])

In [73]:
df_phil_mode.head()

,Calendar_Year,Calendar_Month,Mode,Average_Daily_Ridership,city
0,2019,1,Bus,459160,Philadelphia
2,2019,1,Heavy Rail,296709,Philadelphia
3,2019,1,Regional Rail,122856,Philadelphia
6,2019,2,Bus,464674,Philadelphia
8,2019,2,Heavy Rail,315858,Philadelphia


In [74]:
colonnes_a_supprimer = ['ObjectId', 'Source']
df_phil_route = df_phil_route.drop(columns=[c for c in colonnes_a_supprimer if c in df_phil_route.columns])

In [75]:
df_phil_route.head()

,Calendar_Year,Calendar_Month,Route,Average_Daily_Ridership,city
0,2019,8,59,3811,Philadelphia
1,2019,8,60,8550,Philadelphia
2,2019,8,61,3563,Philadelphia
3,2019,8,62,226,Philadelphia
4,2019,8,64,4335,Philadelphia


In [ ]:
df_phil_route.to_csv(r"C:\Python_challenges\Brief_jury_blanc\output\philadelphia_ridership_by_route_clean.csv")
df_phil_mode.to_csv(r"C:\Python_challenges\Brief_jury_blanc\output\philadelphia_ridership_by_mode_clean.csv")

### Harmonisation Chicago et Philadelphia

##### By mode

In [77]:
df_chicago_mode.head()

,service_date,day_type,bus,rail_boardings,total_rides,year,month
6574,2019-01-01,U,248879,245852,494731,2019,1
6575,2019-01-02,W,591006,573542,1164548,2019,1
6576,2019-01-03,W,664442,627781,1292223,2019,1
6577,2019-01-04,W,668812,628514,1297326,2019,1
6578,2019-01-05,A,444434,348257,792691,2019,1


In [78]:
# Passage en format LONG (bus / rail)
df_chicago_mode = df_chicago_mode.melt(
    id_vars=[ "year", "month"],
    value_vars=["bus", "rail_boardings"],
    var_name="mode",
    value_name="ridership"
)


df_chicago_mode.head()

,year,month,mode,ridership
0,2019,1,bus,248879
1,2019,1,bus,591006
2,2019,1,bus,664442
3,2019,1,bus,668812
4,2019,1,bus,444434


In [79]:
df_chicago_mode = df_chicago_mode.rename(columns={
    "mode": "Mode",
})

df_chicago_mode = (
    df_chicago_mode
    .groupby(["year", "month", "Mode"], as_index=False)
    .agg(ridership=("ridership", "mean"))
)

df_chicago_mode = df_chicago_mode.sort_values(
    ["Mode", "year", "month"]
)

df_chicago_mode["city"] = "Chicago"

df_chicago_mode.head()

,year,month,Mode,ridership,city
0,2019,1,bus,563028.000000,Chicago
2,2019,2,bus,652809.857143,Chicago
4,2019,3,bus,653778.935484,Chicago
6,2019,4,bus,664562.666667,Chicago
8,2019,5,bus,682092.516129,Chicago


In [80]:
df_chicago_mode['Mode'].value_counts()

bus               83
rail_boardings    83
Name: Mode, dtype: int64

In [81]:
df_phil_mode['Mode'].value_counts()

Bus              82
Heavy Rail       82
Regional Rail    82
Name: Mode, dtype: int64

In [82]:
df_phil_mode.head()

,Calendar_Year,Calendar_Month,Mode,Average_Daily_Ridership,city
0,2019,1,Bus,459160,Philadelphia
2,2019,1,Heavy Rail,296709,Philadelphia
3,2019,1,Regional Rail,122856,Philadelphia
6,2019,2,Bus,464674,Philadelphia
8,2019,2,Heavy Rail,315858,Philadelphia


In [83]:
df_phil_mode = df_phil_mode.rename(columns={
    "Calendar_Year": "year",
    "Calendar_Month": "month",
    "Average_Daily_Ridership": "ridership"
})

df_phil_mode['Mode']= df_phil_mode['Mode'].replace({
    "Heavy Rail": "rail_boardings",
    "Regional Rail": "rail_boardings",
    "Bus": "bus"
})

df_phil_mode = (
    df_phil_mode
    .groupby(['year', 'month', 'Mode'], as_index=False)
    .agg({'ridership': 'sum'})
)

df_phil_mode["city"] = "Philadelphia"

In [84]:
df_phil_mode.head()

,year,month,Mode,ridership,city
0,2019,1,bus,459160,Philadelphia
1,2019,1,rail_boardings,419565,Philadelphia
2,2019,2,bus,464674,Philadelphia
3,2019,2,rail_boardings,437019,Philadelphia
4,2019,3,bus,478487,Philadelphia


In [85]:
df_comparative_mode = pd.concat(
    [
        df_phil_mode[["city", "year", "month", "ridership","Mode"]],
        df_chicago_mode[["city", "year", "month", "ridership","Mode"]]
    ],
    ignore_index=True
)

df_comparative_mode.head()

,city,year,month,ridership,Mode
0,Philadelphia,2019,1,459160.0,bus
1,Philadelphia,2019,1,419565.0,rail_boardings
2,Philadelphia,2019,2,464674.0,bus
3,Philadelphia,2019,2,437019.0,rail_boardings
4,Philadelphia,2019,3,478487.0,bus


In [ ]:
df_comparative_mode.to_csv(r"C:\Python_challenges\Brief_jury_blanc\output\comparative_mode_ridership_clean.csv")

#### By route

In [87]:
df_chicago_route.head() 

,date,daytype,rides,route,Calendar_Month
0,2019-01-01,U,3562,47,1
1,2019-01-02,W,6597,47,1
2,2019-01-03,W,7768,47,1
3,2019-01-04,W,8721,47,1
4,2019-01-05,A,6312,47,1


In [88]:
# ==============================
# CHICAGO ROUTE : NORMALISATION
# ==============================

df_chicago_route["date"] = pd.to_datetime(df_chicago_route["date"])
df_chicago_route["Calendar_Year"] = df_chicago_route["date"].dt.year
df_chicago_route["Calendar_Month"] = df_chicago_route["date"].dt.month
df_chicago_route["city"] = "Chicago"

df_chicago_route = df_chicago_route.rename(columns={
    "rides": "ridership"
})

df_chicago_route

df_chicago_route.head()


,date,daytype,ridership,route,Calendar_Month,Calendar_Year,city
0,2019-01-01,U,3562,47,1,2019,Chicago
1,2019-01-02,W,6597,47,1,2019,Chicago
2,2019-01-03,W,7768,47,1,2019,Chicago
3,2019-01-04,W,8721,47,1,2019,Chicago
4,2019-01-05,A,6312,47,1,2019,Chicago


In [89]:
# ==========================================
# CHICAGO : MOYENNE JOURNALIÈRE PAR MOIS
# ==========================================

df_chicago_route = (
    df_chicago_route
    .groupby(["city", "Calendar_Year", "Calendar_Month", "route"], as_index=False)
    .agg(
        ridership=("ridership", "mean")  
    )
)

df_chicago_route = df_chicago_route.rename(columns={
    "ridership": "Average_Daily_Ridership",
    "route":"Route"
})

df_chicago_route.head()


,city,Calendar_Year,Calendar_Month,Route,Average_Daily_Ridership
0,Chicago,2019,1,3,11428.580645
1,Chicago,2019,1,4,14214.419355
2,Chicago,2019,1,30,2127.935484
3,Chicago,2019,1,31,390.363636
4,Chicago,2019,1,34,3438.677419


In [90]:
df_phil_route.head()

,Calendar_Year,Calendar_Month,Route,Average_Daily_Ridership,city
0,2019,8,59,3811,Philadelphia
1,2019,8,60,8550,Philadelphia
2,2019,8,61,3563,Philadelphia
3,2019,8,62,226,Philadelphia
4,2019,8,64,4335,Philadelphia


In [91]:
df_phil_route.shape

(10994, 5)

In [92]:
df_chicago_route.shape

(10466, 5)

In [93]:
df_comparative_route = pd.concat(
    [df_phil_route,df_chicago_route],
    ignore_index=True
)

df_comparative_route.head()

,Calendar_Year,Calendar_Month,Route,Average_Daily_Ridership,city
0,2019,8,59,3811.0,Philadelphia
1,2019,8,60,8550.0,Philadelphia
2,2019,8,61,3563.0,Philadelphia
3,2019,8,62,226.0,Philadelphia
4,2019,8,64,4335.0,Philadelphia


In [ ]:
df_comparative_route.to_csv(r"C:\Python_challenges\Brief_jury_blanc\output\comparative_route_ridership_clean.csv")